In [1]:
import sqlalchemy 
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, MetaData, inspect, func, Column, Integer, String, Numeric, Text, Float
from sqlalchemy.ext.declarative import declarative_base

import pandas as pd

In [2]:
#Step 1. `# Create a Database Connection  
engine = create_engine("sqlite:///hawaii.sqlite", echo=False)

In [3]:
#Step 2. 
# Create a connection to the engine called `conn`
conn = engine.connect()

In [4]:
#Step 3
# Define the Measurement class
# Then your tables 
#`class Measurement(Base):  __tablename__ = "measurements"`
#`class Stations(Base):  __tablename__ = "stations"`

Base = declarative_base()

class Measurement(Base):
    __tablename__ = 'measurement'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(Text)
    prcp = Column(Float)
    tobs = Column(Integer)
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
    
    

In [5]:
class Station(Base):
    __tablename__ = 'station'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    name = Column(Text)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    def __repr__(self):
        return f"id={self.id}, name={self.name}"


In [6]:
#Step 4.
# Use `create_all` to create the table in the database

Base.metadata.create_all(engine)

In [7]:
#Step 5. # Read CSV files into a pandas DataFrame

stations = pd.read_csv("clean_hawaii_stations.csv")

measurements = pd.read_csv("clean_hawaii_measurements.csv")


In [8]:
#Step 6. # Use Orient=‘records’ to create a list of data to write (edited)

stations_data = stations.to_dict(orient='records')

measurements_data = measurements.to_dict(orient='records')


In [9]:
#Step 7. 
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [10]:
#Step 8. 
# Save the reference to the `measurement` table as a variable called `table`
# Bring the table into python using SQLAlchemy
measurement_table = sqlalchemy.Table("measurement", metadata, autoload=True)


In [11]:
# Use `table.delete()` to remove any existing data.
conn.execute(measurement_table.delete())

In [12]:
# Use `table.insert()` to insert the data into the table
conn.execute(measurement_table.insert(), measurements_data)

In [13]:
#same things as in Step 8 for the stations table
station_table = sqlalchemy.Table("station", metadata, autoload=True)

In [14]:
conn.execute(station_table.delete())

In [15]:
conn.execute(station_table.insert(), stations_data)

In [16]:
# Test that the insert works by fetching the first 10 rows for the measurements table. 
conn.execute("select * from measurement limit 10").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-07', 0.06, 70),
 (6, 'USC00519397', '2010-01-08', 0.0, 64),
 (7, 'USC00519397', '2010-01-09', 0.0, 68),
 (8, 'USC00519397', '2010-01-10', 0.0, 73),
 (9, 'USC00519397', '2010-01-11', 0.01, 64),
 (10, 'USC00519397', '2010-01-12', 0.0, 61)]

In [17]:
# Test that the insert works by fetching the first 10 rows for the stations table. 
conn.execute("select * from station limit 10").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 (6, 'USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 (7, 'USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999995, 32.9),
 (8, 'USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 (9, 'USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]

In [18]:
inspector = inspect(engine)
inspector.get_table_names()

['measurement', 'station']